In [31]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
Libraries imported.


In [0]:
df_1 = pd.read_html('http://www.fallingrain.com/world/UK/B8/')

In [0]:
df_pre = df_1[0]

In [50]:
df = df_pre[:16]
df.drop(['What', 'Country', 'Elev ft.', 'Pop est'], axis = 1, inplace = True)
df.columns = ['Neighborhood', 'Borough', 'Lat', 'Long']
df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Neighborhood,Borough,Lat,Long
0,Beckenham,Bromley,51.41,-0.04
1,Bickley,Bromley,51.38,0.03
2,Bromley,Bromley,51.40,0.05
3,Bromley kerület,Bromley,51.40,0.05
4,Chelsfield,Bromley,51.35,0.13
5,Chislehurst,Bromley,51.40,0.07
6,Cudham,Bromley,51.30,0.07
7,Downe,Bromley,51.32,0.05
8,Elmers End,Bromley,51.40,-0.05
9,Farnborough,Bromley,51.35,0.07


In [51]:
address = 'Bromley'

geolocator = Nominatim(user_agent="bromley_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bromley are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bromley are 51.4028046, 0.0148142.


In [59]:
# create map of Bromley using latitude and longitude values
map_bromley = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Lat'], df['Long'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bromley)  
    
map_bromley

In [60]:
CLIENT_ID = 'ZIC52CMGLUGR0QAASNQGEQ0K0RIKVLUAU2GHAQYFMPJFLJTN' # your Foursquare ID
CLIENT_SECRET = 'TLODYFWI0R3CMXZE42CKHVXLTK5UJEWCMWRN4BUHUBVM0CCJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZIC52CMGLUGR0QAASNQGEQ0K0RIKVLUAU2GHAQYFMPJFLJTN
CLIENT_SECRET:TLODYFWI0R3CMXZE42CKHVXLTK5UJEWCMWRN4BUHUBVM0CCJ


In [0]:
df.loc[0, 'Neighborhood']
neighborhood_latitude = df.loc[0, 'Lat']
neighborhood_longitude = df.loc[0, 'Long']

In [0]:
# Getting top 100 venues in Beckenham.

import requests
limit = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    limit)

results = requests.get(url).json()

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [90]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,500º Degrees Pizzeria,Pizza Place,51.408569,-0.038808
1,Coffee Cup House,Coffee Shop,51.408995,-0.041015
2,The Spa at Beckenham,Gym / Fitness Center,51.408821,-0.037682
3,Sainsbury's Local,Grocery Store,51.409039,-0.041880
4,Morley's,Fried Chicken Joint,51.408907,-0.040886


In [91]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

16 venues were returned by Foursquare.


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [82]:
bromley_venues = getNearbyVenues(names= df['Neighborhood'],
                                   latitudes= df['Lat'],
                                   longitudes= df['Long']
                                  )

bromley_venues.head()

Beckenham
Bickley
Bromley
Bromley kerület
Chelsfield
Chislehurst
Cudham
Downe
Elmers End
Farnborough
Foots Cray
Hayes
Keston
Orpington
Saint Mary Cray
West Wickham


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Beckenham,51.41,-0.04,500º Degrees Pizzeria,51.408569,-0.038808,Pizza Place
1,Beckenham,51.41,-0.04,Coffee Cup House,51.408995,-0.041015,Coffee Shop
2,Beckenham,51.41,-0.04,The Spa at Beckenham,51.408821,-0.037682,Gym / Fitness Center
3,Beckenham,51.41,-0.04,Sainsbury's Local,51.409039,-0.041880,Grocery Store
4,Beckenham,51.41,-0.04,Morley's,51.408907,-0.040886,Fried Chicken Joint


In [92]:
#Analyzing each neighborhood
# one hot encoding
bromley_onehot = pd.get_dummies(bromley_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bromley_onehot['Neighborhood'] = bromley_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bromley_onehot.columns[-1]] + list(bromley_onehot.columns[:-1])
bromley_onehot = bromley_onehot[fixed_columns]

bromley_onehot.head()

,Neighborhood,Beer Store,Boutique,Breakfast Spot,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Cricket Ground,Farm,Fish & Chips Shop,Fried Chicken Joint,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Health & Beauty Service,Locksmith,Park,Pizza Place,Platform,Pub,Rental Car Location,Shopping Plaza,Soccer Field,Sports Club,Stables,Supermarket,Tapas Restaurant,Thai Restaurant,Train Station,Tram Station
0,Beckenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Beckenham,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Beckenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Beckenham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Beckenham,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [93]:
bromley_onehot.shape

(60, 35)

In [94]:
bromley_grouped = bromley_onehot.groupby('Neighborhood').mean().reset_index()
bromley_grouped

,Neighborhood,Beer Store,Boutique,Breakfast Spot,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Coffee Shop,Construction & Landscaping,Cricket Ground,Farm,Fish & Chips Shop,Fried Chicken Joint,Gastropub,Golf Course,Grocery Store,Gym / Fitness Center,Health & Beauty Service,Locksmith,Park,Pizza Place,Platform,Pub,Rental Car Location,Shopping Plaza,Soccer Field,Sports Club,Stables,Supermarket,Tapas Restaurant,Thai Restaurant,Train Station,Tram Station
0,Beckenham,0.0625,0.0,0.0625,0.0,0.0,0.000000,0.0,0.0625,0.125,0.000000,0.00,0.00,0.0625,0.0625,0.0,0.000000,0.0625,0.0625,0.0,0.00,0.062500,0.125,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0625,0.0,0.125,0.0625
1,Bickley,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0000,0.000,0.000000,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.0000,0.0000,0.0,0.00,0.000000,0.000,0.0,0.50,0.00,0.00,0.0,0.5,0.000000,0.0,0.0000,0.0,0.000,0.0000
2,Bromley,0.0000,0.0,0.0000,0.0,0.5,0.000000,0.0,0.0000,0.000,0.000000,0.25,0.00,0.0000,0.0000,0.0,0.000000,0.0000,0.0000,0.0,0.00,0.000000,0.000,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0000,0.0,0.250,0.0000
3,Bromley kerület,0.0000,0.0,0.0000,0.0,0.5,0.000000,0.0,0.0000,0.000,0.000000,0.25,0.00,0.0000,0.0000,0.0,0.000000,0.0000,0.0000,0.0,0.00,0.000000,0.000,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0000,0.0,0.250,0.0000
4,Chelsfield,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0000,0.000,0.000000,0.00,0.25,0.0000,0.0000,0.0,0.250000,0.0000,0.0000,0.0,0.00,0.000000,0.000,0.0,0.00,0.25,0.00,0.0,0.0,0.000000,0.0,0.0000,0.0,0.250,0.0000
5,Chislehurst,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0000,0.000,0.000000,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.0000,0.0000,0.0,0.00,0.000000,0.000,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0000,0.0,1.000,0.0000
6,Downe,0.0000,0.0,0.0000,0.0,0.0,0.000000,1.0,0.0000,0.000,0.000000,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.0000,0.0000,0.0,0.00,0.000000,0.000,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0000,0.0,0.000,0.0000
7,Elmers End,0.0000,0.0,0.1000,0.1,0.0,0.000000,0.0,0.0000,0.100,0.000000,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.0000,0.0000,0.0,0.00,0.000000,0.000,0.1,0.10,0.00,0.00,0.1,0.0,0.000000,0.1,0.0000,0.1,0.100,0.1000
8,Farnborough,0.0000,0.0,0.0000,0.0,0.0,0.333333,0.0,0.0000,0.000,0.000000,0.00,0.00,0.0000,0.0000,0.0,0.333333,0.0000,0.0000,0.0,0.00,0.333333,0.000,0.0,0.00,0.00,0.00,0.0,0.0,0.000000,0.0,0.0000,0.0,0.000,0.0000
9,Foots Cray,0.0000,0.0,0.0000,0.0,0.0,0.000000,0.0,0.0000,0.000,0.250000,0.00,0.00,0.0000,0.0000,0.0,0.000000,0.2500,0.0000,0.0,0.00,0.000000,0.000,0.0,0.25,0.00,0.25,0.0,0.0,0.000000,0.0,0.0000,0.0,0.000,0.0000


In [95]:
bromley_grouped.shape

(15, 35)

In [96]:
num_top_venues = 5

for hood in bromley_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bromley_grouped[bromley_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Beckenham----
               venue  freq
0      Train Station  0.12
1        Pizza Place  0.12
2        Coffee Shop  0.12
3         Beer Store  0.06
4  Fish & Chips Shop  0.06


----Bickley----
            venue  freq
0             Pub   0.5
1     Sports Club   0.5
2      Beer Store   0.0
3  Shopping Plaza   0.0
4       Locksmith   0.0


----Bromley----
              venue  freq
0  Business Service  0.50
1    Cricket Ground  0.25
2     Train Station  0.25
3        Beer Store  0.00
4              Park  0.00


----Bromley kerület----
              venue  freq
0  Business Service  0.50
1    Cricket Ground  0.25
2     Train Station  0.25
3        Beer Store  0.00
4              Park  0.00


----Chelsfield----
                 venue  freq
0  Rental Car Location  0.25
1        Train Station  0.25
2          Golf Course  0.25
3                 Farm  0.25
4           Beer Store  0.00


----Chislehurst----
            venue  freq
0   Train Station   1.0
1      Beer Store   0.0
2  Shopping P

In [0]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [98]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bromley_grouped['Neighborhood']

for ind in np.arange(bromley_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bromley_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckenham,Pizza Place,Coffee Shop,Train Station,Tram Station,Breakfast Spot,Chinese Restaurant,Fish & Chips Shop,Fried Chicken Joint,Gym / Fitness Center,Park
1,Bickley,Pub,Sports Club,Tram Station,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Construction & Landscaping,Coffee Shop,Chinese Restaurant
2,Bromley,Business Service,Cricket Ground,Train Station,Tram Station,Construction & Landscaping,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Chinese Restaurant
3,Bromley kerület,Business Service,Cricket Ground,Train Station,Tram Station,Construction & Landscaping,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Chinese Restaurant
4,Chelsfield,Rental Car Location,Golf Course,Farm,Train Station,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Breakfast Spot


In [121]:
#Clustering nto 4 clusters
# set number of clusters
kclusters = 4

bromley_grouped_clustering = bromley_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bromley_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 1, 1, 1, 1, 3, 1, 1, 1], dtype=int32)

In [114]:
# add clustering labels


bromley_merged = df


bromley_merged = bromley_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bromley_merged.head() # check the last columns!

,Neighborhood,Borough,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Beckenham,Bromley,51.41,-0.04,1.0,Pizza Place,Coffee Shop,Train Station,Tram Station,Breakfast Spot,Chinese Restaurant,Fish & Chips Shop,Fried Chicken Joint,Gym / Fitness Center,Park
1,Bickley,Bromley,51.38,0.03,1.0,Pub,Sports Club,Tram Station,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Construction & Landscaping,Coffee Shop,Chinese Restaurant
2,Bromley,Bromley,51.40,0.05,1.0,Business Service,Cricket Ground,Train Station,Tram Station,Construction & Landscaping,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Chinese Restaurant
3,Bromley kerület,Bromley,51.40,0.05,1.0,Business Service,Cricket Ground,Train Station,Tram Station,Construction & Landscaping,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Chinese Restaurant
4,Chelsfield,Bromley,51.35,0.13,1.0,Rental Car Location,Golf Course,Farm,Train Station,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Breakfast Spot


In [0]:
#**Let's see the different clusters**

In [123]:
# 1st Cluster 

bromley_merged.loc[bromley_merged['Cluster Labels'] == 0, bromley_merged.columns[[1] + list(range(5, bromley_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Bromley,Gastropub,Tram Station,Coffee Shop,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Construction & Landscaping,Chinese Restaurant,Train Station


In [124]:
# 2nd Cluster
bromley_merged.loc[bromley_merged['Cluster Labels'] == 1, bromley_merged.columns[[1] + list(range(5, bromley_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bromley,Pizza Place,Coffee Shop,Train Station,Tram Station,Breakfast Spot,Chinese Restaurant,Fish & Chips Shop,Fried Chicken Joint,Gym / Fitness Center,Park
1,Bromley,Pub,Sports Club,Tram Station,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Construction & Landscaping,Coffee Shop,Chinese Restaurant
2,Bromley,Business Service,Cricket Ground,Train Station,Tram Station,Construction & Landscaping,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Chinese Restaurant
3,Bromley,Business Service,Cricket Ground,Train Station,Tram Station,Construction & Landscaping,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Chinese Restaurant
4,Bromley,Rental Car Location,Golf Course,Farm,Train Station,Bus Station,Business Service,Café,Campground,Chinese Restaurant,Breakfast Spot
5,Bromley,Train Station,Tram Station,Coffee Shop,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Construction & Landscaping,Chinese Restaurant
8,Bromley,Tram Station,Bus Station,Train Station,Coffee Shop,Platform,Soccer Field,Pub,Breakfast Spot,Supermarket,Thai Restaurant
9,Bromley,Park,Café,Golf Course,Tram Station,Construction & Landscaping,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Chinese Restaurant
10,Bromley,Grocery Store,Pub,Construction & Landscaping,Shopping Plaza,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Coffee Shop,Chinese Restaurant
11,Bromley,Boutique,Cricket Ground,Tram Station,Coffee Shop,Gastropub,Fried Chicken Joint,Fish & Chips Shop,Farm,Construction & Landscaping,Chinese Restaurant


In [126]:
# 3rd Cluster

bromley_merged.loc[bromley_merged['Cluster Labels'] == 2, bromley_merged.columns[[1] + list(range(5, bromley_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Bromley,Health & Beauty Service,Tram Station,Coffee Shop,Fried Chicken Joint,Fish & Chips Shop,Farm,Cricket Ground,Construction & Landscaping,Chinese Restaurant,Golf Course
